# Project 2: RAG with Atlast Vector Search, LangChain, OpenAI

[Vector Search RAG Tutorial – Combine Your Data with LLMs with Advanced Search around 39:27](https://youtu.be/JEBDfGqrAUA?si=rENOiHcL_PJyVBk6&t=2367)

https://www.mongodb.com/developer/products/atlas/rag-atlas-vector-search-langchain-openai/

In [12]:
from pymongo import MongoClient
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import MongoDBAtlasVectorSearch
from langchain.document_loaders import DirectoryLoader
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
import gradio as gr
from gradio.themes.base import Base
import os

In [3]:
from pathlib import Path
import sys

# Make this path be the project's "base" directory, so we can include modules
notebook_directory_ancestor = Path.cwd().resolve().parent.parent
print(notebook_directory_ancestor)
core_code_directory = notebook_directory_ancestor / "CoreCode/"
data_directory = notebook_directory_ancestor / "ThirdParty" / "unit_tests" / "SampleData/"

is_core_code_directory_in_sys_path = str(core_code_directory) in sys.path
is_notebook_directory_ancestor_in_sys_path = str(notebook_directory_ancestor) in sys.path
print("Is CoreCode directory in sys.path?", is_core_code_directory_in_sys_path)
print("Is notebook directory's ancestor in sys.path?", is_notebook_directory_ancestor_in_sys_path)

/InServiceOfX
Is CoreCode directory in sys.path? False
Is notebook directory's ancestor in sys.path? False


In [4]:
if not is_core_code_directory_in_sys_path:
    sys.path.append(str(core_code_directory))

In [5]:
from CoreCode.MongoDBInterface.connect_to_client import (connect_to_client, GetCollection)
from CoreCode.MongoDBInterface.CreateURI import CreateURI
from CoreCode.Utilities.LoadEnvironmentFile import load_environment_file
load_environment_file()

In [6]:
create_uri = CreateURI()

Enter your password for MongoDB.

In [7]:
new_uri = create_uri.prompt_password()

Enter your password for MongoDB: ········


In [8]:
%%time
client = connect_to_client(new_uri)

CPU times: user 44.6 ms, sys: 30.1 ms, total: 74.8 ms
Wall time: 14.4 s


In [9]:
get_collection = GetCollection("langchain_demo", "collection_of_text_blobs")
collection = get_collection.get_collection(client)

In [10]:
%%time
loader = DirectoryLoader(data_directory, glob="./*.txt", show_progress=True)
data = loader.load()

  0%|                                                     | 0/3 [00:00<?, ?it/s][nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
 33%|███████████████                              | 1/3 [00:23<00:46, 23.11s/it][nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
100%|█████████████████████████████████████████████| 3/3 [00:28<00:00,  9.61s/it]

CPU times: user 2.55 s, sys: 1.04 s, total: 3.59 s
Wall time: 28.8 s


Define the OpenAI Embedding Model we want to use for the source data. The embedding model is different from the language generation model:

In [13]:
openai_api_key = os.environ.get("OPENAI_API_KEY")
embedding_model = OpenAIEmbeddings(openai_api_key=openai_api_key)

/ThirdParty/langchain/libs/core/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [14]:
from langchain_openai import OpenAIEmbeddings

In [15]:
embedding_model = OpenAIEmbeddings(openai_api_key=openai_api_key)

Initialize the VectorStore. Vectorise the text from the documents using the specified embedding model, and insert them into the specified MongoDB collection

In [16]:
%time
vector_store = MongoDBAtlasVectorSearch.from_documents(data, embedding_model, collection=collection )

Start of the [`extract_information.py`](https://youtu.be/JEBDfGqrAUA?si=xVChrbyuLH6J9vRs&t=2780) script.

In [19]:
%time
atlas_vector_store = MongoDBAtlasVectorSearch(collection, embedding_model)

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 10 µs


/ThirdParty/langchain/libs/core/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.vectorstores.mongodb_atlas.MongoDBAtlasVectorSearch` was deprecated in langchain-community 0.0.25 and will be removed in 0.2.0. An updated version of the class exists in the langchain-mongodb package and should be used instead. To use it run `pip install -U langchain-mongodb` and import as `from langchain_mongodb import MongoDBAtlasVectorSearch`.
  warn_deprecated(


In [21]:
# I decided to pip install this here, but not in my Dockerfile. I did
# pip install -U langchain-mongodb
from langchain_mongodb import MongoDBAtlasVectorSearch

In [23]:
%%time
atlas_vector_store = MongoDBAtlasVectorSearch(collection, embedding_model)

CPU times: user 14 µs, sys: 0 ns, total: 14 µs
Wall time: 19.6 µs


In [24]:
openai_api_key = os.environ.get("OPENAI_API_KEY")

In [25]:
def query_data(query):
    # Convert question to vector using OpenAI embeddings
    # Perform Atlas Vector Search using Langchain's vectorStore
    # similarity_search returns MongoDB documents most similar to the query    

    docs = atlas_vector_store.similarity_search(query, K=1)
    as_output = docs[0].page_content

    # Leveraging Atlas Vector Search paired with Langchain's QARetriever

    # Define the LLM that we want to use -- note that this is the Language Generation Model and NOT an Embedding Model
    # If it's not specified (for example like in the code below),
    # then the default OpenAI model used in LangChain is OpenAI GPT-3.5-turbo, as of August 30, 2023
    # temperature=0 means more definitive, less creative.
    llm = OpenAI(openai_api_key=openai_api_key, temperature=0)

    # Get VectorStoreRetriever: Specifically, Retriever for MongoDB VectorStore.
    # Implements _get_relevant_documents which retrieves documents relevant to a query.
    retriever = atlas_vector_store.as_retriever()

    # Load "stuff" documents chain. Stuff documents chain takes a list of documents,
    # inserts them all into a prompt and passes that prompt to an LLM.

    qa = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=retriever)

    # Execute the chain

    retriever_output = qa.run(query)

    # Return Atlas Vector Search output, and output generated using RAG Architecture
    return as_output, retriever_output

In [ ]:
"""

with gr.Blocks(theme=Base(), title="Question Answering App using Vector Search + RAG") as demo:
    gr.Markdown(
        """
        # Question Answering App using Atlas Vector Search + RAG Architecture
        """)
    textbox = gr.Textbox(label="Enter your Question:")
    with gr.Row():
        button = gr.Button("Submit", variant="primary")
    with gr.Column():
        output1 = gr.Textbox(lines=1, max_lines=10, label="Output with just Atlas Vector Search (returns text field as is):")
        output2 = gr.Textbox(lines=1, max_lines=10, label="Output generated by chaining Atlas Vector Search to Langchain's RetrieverQA + OpenAI LLM:")

# Call query_data function upon clicking the Submit button

    button.click(query_data, textbox, outputs=[output1, output2])

demo.launch()
"""

## Exploring the objects we created

In [29]:
example_retriever = atlas_vector_store.as_retriever()

In [30]:
print(type(collection))
print(type(embedding_model))
print(type(atlas_vector_store))
print(type(example_retriever))

<class 'pymongo.collection.Collection'>
<class 'langchain_openai.embeddings.base.OpenAIEmbeddings'>
<class 'langchain_mongodb.vectorstores.MongoDBAtlasVectorSearch'>
<class 'langchain_core.vectorstores.VectorStoreRetriever'>
